This notebook can be run in Colab. 

In [ ]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomCrop(64),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(*stats,inplace=True)
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomCrop(64),
    transforms.ToTensor(),
#     transforms.Normalize(*stats,inplace=True)
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train = ImageFolder("/content/drive/MyDrive/cs480winter2022/5_shot/5_shot/train", 
                    transform = train_transform)
test = ImageFolder("/content/drive/MyDrive/cs480winter2022/5_shot/5_shot/test",
                   transform = test_transform)

In [5]:
random_seed = 480
torch.manual_seed(random_seed);
val_size = int(len(train) * 0.2)
train_size = len(train) - val_size

train_ds, val_ds = random_split(train, [train_size, val_size])
len(train_ds), len(val_ds)

(88, 22)

In [6]:
batch_size = 20

# PyTorch data loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(val_ds, batch_size, num_workers=2, pin_memory=True)

In [7]:
import copy
def train_model(model, criterion, optimizer, num_epochs):
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        
        # train
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in train_dl:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        # validate
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in valid_dl:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
#         if phase == 'train':
#                 scheduler.step()
        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
        print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        print('-' * 30)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
#         scheduler.step(epoch_loss)
    print('Best Val Acc: {:.4f}'.format(best_acc))
    model.load_state_dict(best_model)
    return model

In [ ]:
import torch
import torchvision
import torch.optim as optim
from torch.optim import lr_scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True).to(device)

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(inplace=True),
    nn.Linear(512, 22)).to(device)

criterion = nn.CrossEntropyLoss()
num_epochs = 40
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)
# scheduler = ReduceLROnPlateau(optimizer, 'min')
# train the model
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.01, patience=5)
model = train_model(model, criterion, optimizer, num_epochs)


In [20]:
from torch.utils.data import Dataset
from skimage.io import imread

class readTest(Dataset):
    def __init__(self, images_root):
        self.images_root = images_root
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return 517

    def __getitem__(self, index):
        img_path = f"{self.images_root}/{index}.jpg"
        image = imread(img_path)
        img_tensor = self.transform(image)
        idx = torch.tensor(int(index))
        return (img_tensor, idx)
test_dataset = readTest("/content/drive/MyDrive/cs480winter2022/5_shot/5_shot/test")
test_loader = DataLoader(test_dataset, 32, shuffle=True)

In [ ]:
l = []
for b in test_loader:
    x, y = b
    pred = model(x.to(device))
    preds = pred.argmax(dim=1)
    for i, j in enumerate(preds):
        l.append([y.to(device)[i].item(), j.item()])
l.sort(key=lambda i: i[0])


In [13]:
import pandas as pd
id = [int(i[0]) for i in l]
cat = [int(i[1]) for i in l]
d = train.class_to_idx
d2 = {int(y):int(x) for x,y in d.items()}
output = pd.DataFrame({"id": id, "category": cat})
output = output.replace({"category": d2})
output

,id,category
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
512,512,21
513,513,21
514,514,21
515,515,21


In [ ]:
output.to_csv('output.csv', index=False)